In [1]:
# Establish a connection with an LLM using Groq API key and the `dspy` library 

# pip install dspy-ai python-dotenv
import os
import dspy
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configure DSPy to use Groq with Llama3.3
groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in environment variables")

# Set up the Groq LLM with Llama3.3
llm = dspy.LM(
    model="groq/llama-3.3-70b-versatile",
    api_key=groq_api_key,  # Using Llama3.3 70B model
)

# Set the LLM as the default for DSPy
dspy.settings.configure(lm=llm)


# Define a simple DSPy Signature for question answering
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 5 and 7 words")

# Pass the signature to the ChainOfThought module
generate_answer = dspy.ChainOfThought(BasicQA)

# Call the predictor on a particular input
question = "What is the color of the sky?"
pred = generate_answer(question=question)

print(f"Question: {question}")
print(f"Predicted Answer: {pred.answer}")

/Users/Srikanth/Projects/adu-ai-agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Question: What is the color of the sky?
Predicted Answer: Blue on a sunny day


In [17]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Test dependencies
dependencies = [
    {"group_id": "com.google.guava", "artifact_id": "guava", "homepage": "https://github.com/google/guava"},
    {"group_id": "com.squareup.okhttp3", "artifact_id": "okhttp", "homepage": "https://github.com/square/okhttp"},
    {"group_id": "org.apache.commons", "artifact_id": "commons-lang3", "homepage": "https://commons.apache.org/proper/commons-lang/"},
    {"group_id": "org.hibernate", "artifact_id": "hibernate-core", "homepage": "https://hibernate.org/orm/"},
    {"group_id": "org.flywaydb", "artifact_id": "flyway-core", "homepage": "https://flywaydb.org/"},
    {"group_id": "org.bouncycastle", "artifact_id": "bcprov-jdk15on", "homepage": "https://www.bouncycastle.org/"}
]

# Common changelog filenames for GitHub projects
CHANGELOG_FILENAMES = ["CHANGELOG.md", "CHANGELOG.txt", "docs/changelog.md", "HISTORY.md", "RELEASE-NOTES.md"]

def fetch_dependency_insights(dep):
    dep_name = dep["artifact_id"]
    homepage_url = dep["homepage"]
    group_id = dep["group_id"]
    print(f"\n--- Fetching insights for {dep_name} ---")
    print(f"🌐 Scraping official site: {homepage_url}")

    insights = {}

    # GitHub-based projects
    if "github.com" in homepage_url:
        repo_path = homepage_url.replace("https://github.com/", "").strip("/")
        insights["SECURITY"] = f"https://github.com/{repo_path}/security"
        insights["RELEASE"] = f"https://github.com/{repo_path}/releases"
        insights["MIGRATION"] = f"https://github.com/{repo_path}/wiki/Migration-Guide"

        # Check for changelogs
        for file in CHANGELOG_FILENAMES:
            changelog_url = f"https://github.com/{repo_path}/blob/main/{file}"
            response = requests.head(changelog_url)
            if response.status_code == 200:
                insights["CHANGELOG"] = changelog_url
                break

    else:
        # General scraping for non-GitHub dependencies
        try:
            response = requests.get(homepage_url, timeout=5)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                links = {a.text.strip().lower(): urljoin(homepage_url, a["href"]) for a in soup.find_all("a", href=True)}

                # Match links based on keywords
                keyword_map = {
                    "SECURITY": ["security", "vulnerabilities", "cve"],
                    "RELEASE": ["release", "changelog", "changes"],
                    "MIGRATION": ["migration", "upgrade"]
                }

                for category, keywords in keyword_map.items():
                    for keyword in keywords:
                        for link_text, link_url in links.items():
                            if keyword in link_text:
                                insights[category] = link_url
                                break

        except requests.RequestException:
            print(f"[❌] Failed to fetch {homepage_url}")

    # ✅ **Fallback Mechanism**
    if not insights.get("RELEASE") or not insights.get("SECURITY"):
        fallback_maven_url = f"https://repo1.maven.org/maven2/{group_id.replace('.', '/')}/{dep_name}/"
        print(f"[🔄] Fallback: Checking Maven Repository → {fallback_maven_url}")
        insights["MAVEN_REPO"] = fallback_maven_url

    if dep_name.startswith("bcprov"):  # Special case for BouncyCastle
        insights["DOCS"] = "https://www.bouncycastle.org/documentation.html"

    # Print results
    for category, link in insights.items():
        print(f"[{category.upper()}] {link}")

# Run for all test dependencies
for dep in dependencies:
    fetch_dependency_insights(dep)

print("\n--- Done Testing ---")



--- Fetching insights for guava ---
🌐 Scraping official site: https://github.com/google/guava
[SECURITY] https://github.com/google/guava/security
[RELEASE] https://github.com/google/guava/releases
[MIGRATION] https://github.com/google/guava/wiki/Migration-Guide

--- Fetching insights for okhttp ---
🌐 Scraping official site: https://github.com/square/okhttp
[SECURITY] https://github.com/square/okhttp/security
[RELEASE] https://github.com/square/okhttp/releases
[MIGRATION] https://github.com/square/okhttp/wiki/Migration-Guide

--- Fetching insights for commons-lang3 ---
🌐 Scraping official site: https://commons.apache.org/proper/commons-lang/
[SECURITY] https://commons.apache.org/security.html
[RELEASE] https://commons.apache.org/proper/commons-lang/changes-report.html
[MIGRATION] https://commons.apache.org/proper/commons-lang/article3_0.html

--- Fetching insights for hibernate-core ---
🌐 Scraping official site: https://hibernate.org/orm/
[SECURITY] https://access.redhat.com/security/t